# Validate vulnerability file

Validate the content of a vulnerability file, and plot up the functions for use in reporting. 

In [ ]:
import os
import pandas as pd
from lxml import etree

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context("talk")
sns.set_palette("Paired", 10)

Specify the path where the vulnerability data are stored, and the specific file names. 

In [ ]:
basePath = r"X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\vulnerability"
nrmlFile = os.path.join(basePath, "domestic_wind_2023.xml")
mappingFile = os.path.join(basePath, "NWRA domestic wind 2023 vulnerability attribute mapping table.csv")
ns = {'nrml': 'http://openquake.org/xmlns/nrml/0.5'}


Parse the NRML file, returning a list of the vulnerability function labels that exist in the file.

In [ ]:
nrmltree = etree.parse(nrmlFile)
vulnFuncs = nrmltree.xpath(
    '//nrml:vulnerabilityFunction/@id',
    namespaces=ns
)

Load the mapping table and get a list of the unique vulnerability function labels

In [ ]:
mapdf = pd.read_csv(mappingFile)
mapFuncs = mapdf['Wind Vulnerability Function ID'].unique().tolist()

This prints the set of functions that exist in the NRML file, but do not exist in the mapping file. If the list is empty, then all vulnerability functions from the NRML file are found in the mapping table.

In [ ]:
list(set(vulnFuncs) - set(mapFuncs))

This prints the set of functions that exist in the mapping file, but do not exist in the NRML file. If the list is empty, then all vulnerability functions from the mapping table are found in the NRML file. If both are empty, then the set of vulnerability functions match. 

In [ ]:
list(set(mapFuncs) - set(vulnFuncs))

In [ ]:
vulnFuncs = nrmltree.xpath(
    '//nrml:vulnerabilityFunction',
    namespaces=ns
)

In [ ]:
def plotVulnFunc(id, iml, meanlr):
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    iml *= 3.6 # Convert to km/h
    ax.plot(iml, meanlr, lw=2, ls='--')
    ax.set_xlabel("0.2 second gust wind speed [km/h]")
    ax.set_ylabel("Damage ratio")
    ax.set_ylim((0, 1.0))
    ax.set_title(f"Vulnerability function: {id}")
    fig.tight_layout()
    plt.savefig(os.path.join(basePath, "plots", f"{id}.png"))
    plt.close(fig)

In [ ]:
for vf in vulnFuncs:
    vfid = vf.get('id')
    imls = vf.find('./nrml:imls', namespaces=ns)
    meanLRs = vf.find('./nrml:meanLRs', namespaces=ns)
    if imls is not None and meanLRs is not None:
        imls_values = np.array([float(value) for value in imls.text.strip().split()])
        meanLRs_values = np.array([float(value) for value in meanLRs.text.strip().split()])
        plotVulnFunc(vfid, imls_values, meanLRs_values)
        #print(f"vulnerabilityFunction ID: {vfid}")
        #print(f"imls values: {imls_values}")
        #print(f"meanLRs values: {meanLRs_values}")
        #print()

In [ ]:
for idx, grp in mapdf.groupby(['State', 'Region', 'AS4055  Class', 'Age', 'Roof material']):
    wvfid = grp['Wind Vulnerability Function ID'].values
    labels = grp['Wall material'].values
    title = f"{idx[0]} - {idx[1]} - {idx[2]} - {idx[3]} - {idx[4]}"
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    for id, label in zip(wvfid, labels):
        targetvf = nrmltree.xpath(f'//nrml:vulnerabilityFunction[@id="{id}"]', namespaces=ns)
        imls = targetvf[0].find('./nrml:imls', namespaces=ns)
        meanLRs = targetvf[0].find('./nrml:meanLRs', namespaces=ns)
        if imls is not None and meanLRs is not None:
            imls_values = np.array([float(value) for value in imls.text.strip().split()])
            meanLRs_values = np.array([float(value) for value in meanLRs.text.strip().split()])
        else:
            next
        imls_values *= 3.6 # Convert to km/h
        ax.plot(imls_values, meanLRs_values, label=label, lw=2, ls='--')
    ax.set_xlabel("0.2 second gust wind speed [km/h]")
    ax.set_ylabel("Damage ratio")
    ax.set_ylim((0, 1.0))
    ax.set_title(title)
    ax.legend(fontsize='x-small', title="Wall material")
    fig.tight_layout()
    plt.savefig(os.path.join(basePath, "plots", f"{title.replace('/', '-')}.png"))
    plt.close(fig)

In [ ]:
exceldf = pd.read_excel(r"X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\vulnerability\ACS Phase 0 wind vulnerability curves.xlsx", sheet_name="NWRA", header=4,)
exceldf.set_index("Model Number", drop=False, inplace=True)
exceldf.drop('Unnamed: 8', axis=1, inplace=True)
exceldf.dropna(subset='Alpha', inplace=True)
imls = exceldf.columns[-38:].values

In [ ]:
for idx, grp in exceldf.groupby(['Region', 'Age', 'AS4055 class.', 'Roof material'], dropna=False):
    wvfid = grp['Model Number'].values
    labels = grp['Wall material'].values
    if pd.isna(idx[2]):
        title = f"{idx[0]} - {idx[1]} - {idx[3]}"
    else:
        title = f"{idx[0]} - {idx[1]} - {idx[2]} - {idx[3]}"
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    for id, label in zip(wvfid, labels):
        meanLRs = exceldf.loc[id].values[-38:]
        ax.plot(imls*3.6, meanLRs, label=label, lw=2, ls='--')
    ax.set_xlabel("0.2 second gust wind speed [km/h]")
    ax.set_ylabel("Damage ratio")
    ax.set_ylim((0, 1.0))
    ax.set_title(title)
    ax.legend(fontsize='x-small', title="Wall material")
    fig.tight_layout()
    plt.savefig(os.path.join(basePath, "plots", f"{title.replace('/', '-')}.png"))
    plt.close(fig)